# Exchange Simulation and Agent-Based Modeling (ABM)
## Price-Time Priority Matching Engine + Multi-Agent Simulation

This notebook demonstrates:
- Exchange-level order matching with price-time priority
- Queue position tracking
- Agent-based modeling with:
  - Informed traders (trade based on true value)
  - Noise traders (random trading)
  - Market makers (inventory management)
- Order book dynamics visualization
- Market microstructure patterns

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import time

from src.config import *
from src.four_exchange_simulator import MatchingEngine, ABMSimulator
from utils.simulation_utils import Order, OrderSide, OrderType, generate_order_id
from utils.plotting_utils import plot_orderbook_snapshot
from utils.io_utils import write_parquet

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Initialize Matching Engine

In [ ]:
# Create matching engine
engine = MatchingEngine(
    instrument_id="SIM_ASSET",
    tick_size=0.01,
    lot_size=1.0,
    enable_self_trade_prevention=True
)

print(f"Matching engine initialized for {engine.instrument_id}")
print(f"Tick size: {engine.tick_size}")
print(f"Lot size: {engine.lot_size}")

## 2. Manual Order Submission Test
### Test basic order matching mechanics

In [ ]:
# Submit limit buy order
buy_order = Order(
    order_id=generate_order_id("TEST"),
    instrument_id="SIM_ASSET",
    side=OrderSide.BUY,
    order_type=OrderType.LIMIT,
    price=99.99,
    quantity=100,
    timestamp=pd.Timestamp.now(),
    trader_id="MANUAL_1"
)

fills = engine.match_order(buy_order)
print(f"Buy order submitted: {buy_order.order_id}")
print(f"Fills: {len(fills)}")
print(f"Best bid: {engine.get_best_bid()}")

In [ ]:
# Submit limit sell order
sell_order = Order(
    order_id=generate_order_id("TEST"),
    instrument_id="SIM_ASSET",
    side=OrderSide.SELL,
    order_type=OrderType.LIMIT,
    price=100.01,
    quantity=100,
    timestamp=pd.Timestamp.now(),
    trader_id="MANUAL_2"
)

fills = engine.match_order(sell_order)
print(f"Sell order submitted: {sell_order.order_id}")
print(f"Fills: {len(fills)}")
print(f"Best ask: {engine.get_best_ask()}")
print(f"Mid price: {engine.get_mid_price()}")

In [ ]:
# Test matching: aggressive buy order
aggressive_buy = Order(
    order_id=generate_order_id("TEST"),
    instrument_id="SIM_ASSET",
    side=OrderSide.BUY,
    order_type=OrderType.LIMIT,
    price=100.01,  # Crosses spread
    quantity=50,
    timestamp=pd.Timestamp.now(),
    trader_id="MANUAL_3"
)

fills = engine.match_order(aggressive_buy)
print(f"Aggressive buy submitted")
print(f"Number of fills: {len(fills)}")
if fills:
    print(f"Fill price: {fills[0].price}, quantity: {fills[0].quantity}")
print(f"Total trades executed: {engine.total_trades}")
print(f"Total volume: {engine.total_volume}")

## 3. Visualize Current Order Book State

In [ ]:
# Get order book snapshot
snapshot = engine.get_order_book_snapshot(n_levels=10)

# Extract bid/ask data
bid_prices = [snapshot.get(f'bid_px_{i}', 0) for i in range(1, 11)]
bid_sizes = [snapshot.get(f'bid_sz_{i}', 0) for i in range(1, 11)]
ask_prices = [snapshot.get(f'ask_px_{i}', 0) for i in range(1, 11)]
ask_sizes = [snapshot.get(f'ask_sz_{i}', 0) for i in range(1, 11)]

# Filter out zeros
bid_prices = [p for p in bid_prices if p > 0]
bid_sizes = bid_sizes[:len(bid_prices)]
ask_prices = [p for p in ask_prices if p > 0]
ask_sizes = ask_sizes[:len(ask_prices)]

if bid_prices and ask_prices:
    fig = plot_orderbook_snapshot(
        np.array(bid_prices),
        np.array(bid_sizes),
        np.array(ask_prices),
        np.array(ask_sizes),
        title="Current Order Book State"
    )
    plt.show()
else:
    print("Order book is empty or incomplete")

## 4. Agent-Based Modeling Simulation
### Initialize ABM with multiple trader types

In [ ]:
# Create fresh matching engine for ABM
engine_abm = MatchingEngine(
    instrument_id="SIM_ASSET",
    tick_size=0.01,
    lot_size=1.0
)

# Initialize ABM simulator
abm = ABMSimulator(
    matching_engine=engine_abm,
    n_informed=5,      # 5 informed traders
    n_noise=20,        # 20 noise traders
    n_mm=3             # 3 market makers
)

print(f"ABM initialized with:")
print(f"  - {abm.n_informed} informed traders")
print(f"  - {abm.n_noise} noise traders")
print(f"  - {abm.n_mm} market makers")
print(f"\nTrue value (known to informed traders): {abm.true_value:.2f}")

## 5. Run Short Simulation

In [ ]:
# Run simulation for 30 seconds
print("Running ABM simulation...")
df_sim = abm.run_simulation(
    duration_seconds=30,
    time_step_ms=100
)

print(f"\nSimulation complete!")
print(f"Generated {len(df_sim)} order book snapshots")
print(f"Total trades executed: {engine_abm.total_trades}")
print(f"Total volume traded: {engine_abm.total_volume:.0f}")
print(f"\nFinal mid price: {df_sim['mid_price'].iloc[-1]:.4f}")
print(f"Price range: [{df_sim['mid_price'].min():.4f}, {df_sim['mid_price'].max():.4f}]")

## 6. Analyze Simulation Results

In [ ]:
# Price evolution
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Mid price
axes[0].plot(df_sim['timestamp'], df_sim['mid_price'], linewidth=1, label='Mid Price')
axes[0].axhline(abm.true_value, color='red', linestyle='--', label='True Value', alpha=0.7)
axes[0].set_ylabel('Price')
axes[0].set_title('Mid Price Evolution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Spread
df_sim['spread'] = df_sim['ask_px_1'] - df_sim['bid_px_1']
axes[1].plot(df_sim['timestamp'], df_sim['spread'], linewidth=1, color='orange')
axes[1].set_ylabel('Spread')
axes[1].set_title('Bid-Ask Spread')
axes[1].grid(True, alpha=0.3)

# Depth
axes[2].plot(df_sim['timestamp'], df_sim['bid_sz_1'], label='Bid Size', alpha=0.7)
axes[2].plot(df_sim['timestamp'], df_sim['ask_sz_1'], label='Ask Size', alpha=0.7)
axes[2].set_ylabel('Size')
axes[2].set_xlabel('Time')
axes[2].set_title('Top of Book Liquidity')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Market Maker Inventory Evolution

In [ ]:
# Plot market maker inventories
mm_inventories = abm.mm_inventories

plt.figure(figsize=(10, 6))
mm_ids = list(mm_inventories.keys())
mm_values = list(mm_inventories.values())

plt.bar(mm_ids, mm_values, color='steelblue', alpha=0.7)
plt.axhline(0, color='black', linestyle='-', linewidth=0.8)
plt.ylabel('Inventory Position')
plt.xlabel('Market Maker ID')
plt.title('Final Market Maker Inventory Positions')
plt.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("Market Maker Inventories:")
for mm_id, inventory in mm_inventories.items():
    print(f"  {mm_id}: {inventory:.0f} shares")

## 8. Price Discovery Analysis

In [ ]:
# Analyze how price converges to true value
df_sim['price_error'] = df_sim['mid_price'] - abm.true_value
df_sim['abs_price_error'] = df_sim['price_error'].abs()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Price error over time
ax1.plot(df_sim['timestamp'], df_sim['price_error'], linewidth=1)
ax1.axhline(0, color='red', linestyle='--', alpha=0.5)
ax1.set_xlabel('Time')
ax1.set_ylabel('Price Error (Mid - True Value)')
ax1.set_title('Price Discovery: Error Over Time')
ax1.grid(True, alpha=0.3)

# Rolling absolute error
ax2.plot(df_sim['timestamp'], df_sim['abs_price_error'].rolling(10).mean(), linewidth=2)
ax2.set_xlabel('Time')
ax2.set_ylabel('Absolute Price Error (10-period MA)')
ax2.set_title('Price Discovery: Convergence')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean absolute price error: {df_sim['abs_price_error'].mean():.4f}")
print(f"Final price error: {df_sim['price_error'].iloc[-1]:.4f}")

## 9. Trade Statistics

In [ ]:
# Analyze fills
fills = engine_abm.fills

if len(fills) > 0:
    fills_df = pd.DataFrame([{
        'timestamp': f.timestamp,
        'price': f.price,
        'quantity': f.quantity,
        'side': f.side.value
    } for f in fills])
    
    print("Trade Statistics:")
    print(f"Total trades: {len(fills_df)}")
    print(f"Total volume: {fills_df['quantity'].sum():.0f}")
    print(f"Average trade size: {fills_df['quantity'].mean():.1f}")
    print(f"Price range: [{fills_df['price'].min():.4f}, {fills_df['price'].max():.4f}]")
    print(f"\nBuy trades: {(fills_df['side'] == 'B').sum()}")
    print(f"Sell trades: {(fills_df['side'] == 'S').sum()}")
    
    # Trade size distribution
    plt.figure(figsize=(10, 5))
    plt.hist(fills_df['quantity'], bins=30, alpha=0.7, edgecolor='black')
    plt.xlabel('Trade Size')
    plt.ylabel('Frequency')
    plt.title('Trade Size Distribution')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("No trades executed during simulation")

## 10. Save Simulation Results

In [ ]:
# Save simulation data
output_file = SIMULATION_PATH / "abm_simulation_notebook.parquet"
write_parquet(df_sim, output_file)
print(f"Simulation results saved to: {output_file}")

# Summary statistics
summary = {
    'duration_seconds': 30,
    'n_snapshots': len(df_sim),
    'n_trades': engine_abm.total_trades,
    'total_volume': engine_abm.total_volume,
    'initial_price': df_sim['mid_price'].iloc[0],
    'final_price': df_sim['mid_price'].iloc[-1],
    'true_value': abm.true_value,
    'mean_spread': df_sim['spread'].mean(),
    'price_volatility': df_sim['mid_price'].std()
}

print("\nSimulation Summary:")
for key, value in summary.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

## 11. Conclusions

Key observations from the ABM simulation:

1. **Price Discovery**: The mid price converges toward the true value as informed traders exploit mispricings
2. **Market Making**: Market makers provide liquidity and manage inventory risk
3. **Spread Dynamics**: Spread widens during high volatility and tightens with market maker competition
4. **Order Flow**: Diverse trader types create realistic order flow patterns
5. **Matching Mechanics**: Price-time priority ensures fair order execution

This simulation framework can be used to:
- Test trading strategies in controlled environments
- Validate model predictions against simulated markets
- Understand market microstructure mechanics
- Assess strategy robustness across different market conditions